# Next Word Prediction:

### Importing The Required Libraries:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import os.path


In [2]:

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


In [3]:
filename = "lazar"
exist = os.path.isfile("corpus-"+filename+"-cleaned.txt")
if not exist:
    file = open(filename + ".txt", "r", encoding = "utf8")
    lines = []
    l = 0
    for i in file:
        lines.append(i)
        l = l + 1
    if (l > 5):
        print("The First Line: ", lines[0])
        print("The Last Line: ", lines[-1])
else:
    file = open("corpus-"+filename+"-cleaned.txt", "r", encoding = "utf8")
    lines = []
    for i in file:
        lines.append(i)
    data = lines[0]

The First Line:   No, like I said, I'm not going to really concern myself with that. Not till later.

The Last Line:   From his mom...


### Cleaning the data:

In [4]:
if not exist:
    data = ""

    for i in lines:
        data = ' '. join(lines)

    data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
    
    
data[:360]

" No, like I said, I'm not going to really concern myself with that. Not till later.  I wasn't told what I was working on...  At that point... no.  Yeah.  The best thing for me to do is to pull out...  I have these calendars... they're big wall calendars where I write what happened every day on it... and I've had them since like 1980... so I have exact dates."

In [5]:
if not exist:
    import string

    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    data = data.translate(translator)

data[:500]

'No like I said m not going to really concern myself with that Not till later wasn t told what was working on At point no Yeah The best thing for me do is pull out have these calendars they re big wall where write happened every day it and ve had them since 1980 so exact dates And photocopied some of those can remember save my life even within a year EG G this would be second one But the first different job Five hesitate because don know One did Dennis who wound up being supervisor still he looke'

In [5]:
if not exist:
    z = []

    for i in data.split():
        if i not in z:
            z.append(i)

    data = ' '.join(z)
data[:500]

"No, like I said, I'm not going to really concern myself with that. Not till later. wasn't told what was working on... At that point... no. Yeah. The best thing for me do is pull out... have these calendars... they're big wall calendars where write happened every day on it... and I've had them since 1980... so exact dates. And photocopied some of those so... can't remember dates save my life... even within a year. EG&G. Yeah, this would be second one. But the first one different job. Five? No. Ye"

In [6]:
if not exist:
    file1 = open("corpus-"+filename+"-cleaned.txt","w", encoding = "utf8")
    file1.write(data)
    file1.close() #to change file access modes

### Tokenization:

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer-'+filename+'.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[4, 49, 1, 105, 1100, 23, 272, 50, 51, 1101]

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

3348


In [9]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  5431


array([[   4,   49],
       [  49,    1],
       [   1,  105],
       [ 105, 1100],
       [1100,   23],
       [  23,  272],
       [ 272,   50],
       [  50,   51],
       [  51, 1101],
       [1101,  273]])

In [10]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [11]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [   4   49    1  105 1100]
The responses are:  [  49    1  105 1100   23]


In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Creating the Model:

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             33480     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 3348)              3351348   
Total params: 16,433,828
Trainable params: 16,433,828
Non-trainable params: 0
_________________________________________________________________


### Plot The Model:

In [15]:

from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='model.png', show_layer_names=True)

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (C:\Users\Platinum\ana3\envs\arioo5\lib\site-packages\tensorflow\python\eager\context.py)

### Callbacks:

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword-"+filename+".h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword'+filename
tensorboard_Visualization = TensorBoard(log_dir=logdir)

### Compile The Model:

In [17]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

### Fit The Model:

In [18]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
5376/5431 [============================>.] - ETA: 0s - loss: 8.1217
Epoch 00001: loss improved from inf to 8.12181, saving model to nextword-lazar.h5
5431/5431 [==============================] - 6s 1ms/sample - loss: 8.1218
Epoch 2/50
5376/5431 [============================>.] - ETA: 0s - loss: 8.0975
Epoch 00002: loss improved from 8.12181 to 8.09731, saving model to nextword-lazar.h5
5431/5431 [==============================] - 2s 300us/sample - loss: 8.0973
Epoch 3/50
5184/5431 [===========================>..] - ETA: 0s - loss: 8.0356
Epoch 00003: loss improved from 8.09731 to 8.03597, saving model to nextword-lazar.h5
5431/5431 [==============================] - 1s 228us/sample - loss: 8.0360
Epoch 4/50
5376/5431 [============================>.] - ETA: 0s - loss: 7.8487
Epoch 00004: loss improved from 8.03597 to 7.84851, saving model to nextword-lazar.h5
5431/5431 [=============

Epoch 00033: loss improved from 2.75585 to 2.71753, saving model to nextword-lazar.h5
5431/5431 [==============================] - 1s 237us/sample - loss: 2.7175
Epoch 34/50
5248/5431 [===========================>..] - ETA: 0s - loss: 2.6233
Epoch 00034: loss improved from 2.71753 to 2.63083, saving model to nextword-lazar.h5
5431/5431 [==============================] - 1s 234us/sample - loss: 2.6308
Epoch 35/50
5312/5431 [============================>.] - ETA: 0s - loss: 2.5671
Epoch 00035: loss improved from 2.63083 to 2.57521, saving model to nextword-lazar.h5
5431/5431 [==============================] - 1s 234us/sample - loss: 2.5752
Epoch 36/50
5376/5431 [============================>.] - ETA: 0s - loss: 2.4976
Epoch 00036: loss improved from 2.57521 to 2.50144, saving model to nextword-lazar.h5
5431/5431 [==============================] - 2s 302us/sample - loss: 2.5014
Epoch 37/50
5376/5431 [============================>.] - ETA: 0s - loss: 2.4361
Epoch 00037: loss improved from 

### Graph:

In [17]:
# https://stackoverflow.com/questions/26649716/how-to-show-pil-image-in-ipython-notebook
# tensorboard --logdir="./logsnextword1"
# http://DESKTOP-U3TSCVT:6006/

#from IPython.display import Image 
#pil_img = Image(filename='graph1.png')
#display(pil_img)

## Observation:
### We are able to develop a decent next word prediction model and are able to get a declining loss and an overall decent performance.

In [92]:
def Predict_Next_Words(model, tokenizer, text):
    #print(text)
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = np.array(sequence)
    res = []
    preds = model.predict(sequence)
    preds1= np.flip(np.argsort(preds,axis=1))
    preds = np.argmax(preds,axis=1)
    res.append((preds1[0][0]))
    res.append((preds1[0][1]))
    res.append((preds1[0][2]))
    print(len(res))
    predicted_word = ""
    predicted_words = []

    for key, value in tokenizer.word_index.items():
        if value in res:
            predicted_words.append(key)
            #print(key)
            #break

    print(len(predicted_word))
    return predicted_words

In [95]:
text = "Obviously";
export = Predict_Next_Words(model, tokenizer, text)
print(export)

3
0
['weapons', 'grill', 'attempt']


In [73]:
c = 0
text = input("Enter your line: ")
sentence = ""
while(c<10):
    sentence = sentence + " " + text

    #if text == "stop":
        #print("Ending The Program.....")
        #break
    
    #else:
    try:
        #text = text.split(" ")
        #text = text[-1]

        #text = ''.join(text)
        text = Predict_Next_Words(model, tokenizer, text)
        c = c + 1
    except:
        continue
print(sentence)

Enter your line: call
1572
3177
1330
[1572]
234
2
51
[234]
784
754
441
[784]
28
601
83
[28]
1976
1649
272
[1976]
79
522
546
[79]
1198
929
1431
[1198]
59
86
250
[59]
170
1107
781
[170]
1969
2060
325
[1969]
 call parasitic guards understand where lunchroom more than them good


In [36]:
A = np.matrix([[1,2,3,33],[88,5,6,66],[7,8,9,99]])

s= np.argmax(A,axis=1)  # 11, which is the position of 99
print(s)

[[3]
 [0]
 [3]]
